In [2]:
!pip3 install pandas

In [4]:
!pip3 install NumPy

In [5]:
!pip3 install matplotlib


In [6]:
!pip3 install seaborn


In [7]:
!pip3 install scikit-learn

In [8]:
!pip3 install textblob


In [12]:
!pip3 install nltk
import ssl
import nltk

ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>


False

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
# Load dataset
data = pd.read_csv("gossipcop_fake.csv")

# Preprocessing
def preprocess_text(text):
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

data['cleaned_text'] = data['title'].apply(preprocess_text)

# Named Entity Recognition (NER) Replacement
def extract_entities_alternative(text):
    words = word_tokenize(text)
    common_entities = {'org': ['company', 'organization', 'corporation'],
                       'gpe': ['city', 'country', 'state'],
                       'person': ['mr', 'mrs', 'dr', 'ms']}
    entities = {'ORG': 0, 'GPE': 0, 'PERSON': 0}
    for word in words:
        lower_word = word.lower()
        if lower_word in common_entities['org']:
            entities['ORG'] += 1
        elif lower_word in common_entities['gpe']:
            entities['GPE'] += 1
        elif lower_word in common_entities['person']:
            entities['PERSON'] += 1
    return entities

entity_features = data['cleaned_text'].apply(extract_entities_alternative)
data = pd.concat([data, pd.DataFrame(entity_features.tolist())], axis=1)

# Additional Features
data['article_length'] = data['cleaned_text'].apply(lambda x: len(x.split()))
data['sentiment'] = data['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
data['engagement_metric'] = data['tweet_ids'].apply(lambda x: len(str(x).split('\t')))

# Feature Engineering
features = ['ORG', 'GPE', 'PERSON', 'article_length', 'sentiment']
X = data[features]
y = data['engagement_metric']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Predictive Modeling
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x=features, y=model.feature_importances_)
plt.title("Feature Importances")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.show()

# Additional Visualizations
sns.heatmap(data[features + ['engagement_metric']].corr(), annot=True, cmap="coolwarm")
plt.title("Feature Correlation Heatmap")
plt.show()

# Save results
data.to_csv("processed_data.csv", index=False)

# Documentation (Skeleton for Report)
with open("report.txt", "w") as f:
    f.write("""Project Report

Methodology:
1. Preprocessed text data to clean and normalize.
2. Extracted named entities using an alternative method and categorized them.
3. Engineered features including article length, sentiment, and entity counts.
4. Built a Random Forest model to predict engagement metrics.
5. Evaluated model performance using MAE and R-squared metrics.

Findings:
- Key insights from feature importance and correlations.
- Recommendations for improving article engagement based on entity analysis.

""")
